In [1]:
import mysql.connector

mydb = mysql.connector.connect(
    host="localhost",
    user="root",
    password="root123"
)

cursor = mydb.cursor()
cursor.execute("CREATE DATABASE RandomX")
# cursor.execute('use randomx;')

In [2]:
import pandas as pd
userdata = pd.read_excel('userdata.xls')
movie_info = pd.read_excel('movie_info.xlsx')
movie_reviews = pd.read_excel('movie_reviews.xls')

In [3]:
cursor.execute('use randomx;')
cursor.execute(''' CREATE TABLE userdata (name VARCHAR(20) PRIMARY KEY NOT NULL, 
                    geneder CHAR(1), age INT); ''')


cursor.execute(''' CREATE TABLE movie_info (movie_id VARCHAR(255) PRIMARY KEY, 
            movie_title VARCHAR(255) NOT NULL, audience_rating NUMERIC(3, 2) ); ''')

cursor.execute(''' CREATE TABLE movie_reviews (movie_id VARCHAR(255) NOT NULL, 
            user_name VARCHAR(20), review VARCHAR(300) NOT NULL,
            FOREIGN KEY (movie_id) REFERENCES  movie_info(movie_id),
            FOREIGN KEY (user_name) REFERENCES  userdata(name),
            PRIMARY KEY(movie_id, user_name, review)
            ); ''')

In [4]:
for i in userdata.index:
    cmd = ''' INSERT INTO userdata VALUES(%s, %s, %s) '''
    val = (userdata['username'][i], userdata['gender'][i], str(userdata['age'][i]))
    cursor.execute(cmd, val)

for i in movie_info.index:
    cmd = ''' INSERT INTO movie_info VALUES(%s, %s, %s)'''
    val = (movie_info['movie_id'][i], movie_info['movie_title'][i], str(movie_info['audience_rating'][i]))
    cursor.execute(cmd, val)
    
for i in movie_reviews.index:
    cmd = ''' INSERT INTO movie_reviews VALUES(%s, %s, %s)'''
    val = (movie_reviews['movie_id'][i], movie_reviews['user_name'][i], movie_reviews['movie_rev'][i])
    cursor.execute(cmd, val)
  
mydb.commit()    

In [5]:
f1 = open("negative-words.txt", "r")
f2 = open("positive-words.txt", "r")
for i in range(35):
    f1.readline()
    f2.readline()

negative_words = f1.read().splitlines()
positive_words = f2.read().splitlines()

In [6]:
cursor.execute("ALTER TABLE movie_reviews ADD sentiment VARCHAR(12)")
mydb.commit()

In [7]:
import re
def sentiment(review):
    review = review.lower()
    review = re.sub('[^0-9a-z-*]', ' ', review)
    review = review.split()
    count_n, count_p = 0, 0
    for  r in review:
        if(r in negative_words):
            count_n+=1
        if(r in positive_words):
            count_p += 1
    if(count_p > count_n):
        return "positive"
    elif(count_n > count_p):
        return "negative"
    else:
        return "neutral"

In [8]:
for i in movie_reviews.index:
    cmd = "select * from movie_reviews where sentiment is NULL limit 1;"
    cursor.execute(cmd)
    x = cursor.fetchall()
    movie_id = x[0][0]
    user_name = x[0][1]
    review = x[0][2]
    senti = sentiment(review)
    cmd = "update movie_reviews set sentiment=%s where movie_id=%s and user_name=%s and review = %s;" 
    cursor.execute(cmd, (senti, movie_id, user_name, review))
    mydb.commit()

In [11]:
cmd = ''' select movie_title as movie_name 
          from movie_info natural join movie_reviews 
          where audience_rating>3.5 and sentiment="positive" 
          group by movie_id 
          having count(sentiment) >= 2 
          order by audience_rating desc 
          limit 5; '''
cursor.execute(cmd)
x=cursor.fetchall()
for foo in x:
    print(foo)

('The Lord of the Rings: The Fellowship of the Ring',)
('Room',)
('The Man Who Shot Liberty Valance',)
('Duck Soup',)
('Run Lola Run',)
